# **Imports**

In [59]:
from math import log2,ceil
import numpy as np

# **Some functions**

**text-to-hex**

In [60]:
def t2h(input):
    orders = [ord(char)-ord('a')  if char != '#' else 26 for char in [char for char in input.lower()] ]
    return ''.join([f'0{hex(num)[2:]}' if num <= 15 else f'{hex(num)[2:]}' for num in orders])

**hex-to-text**

In [61]:
def h2t(text):
    letters = [text[i]+text[i+1] for i in range(0,len(text),2)]
    letters = [int(l,16) for l in letters]
    for i,l in enumerate(letters):
        letters[i] = chr(ord('a')+l) if l != 26 else '#'
    return ''.join(letters)


**hex-to-binary**

In [62]:
def h2b(text):
    return ''.join(['0'*int(4-ceil(log2(int(ch,16))+0.001))+bin(int(ch,16))[2:] if ch != '0' else '0000' for ch in text])
    

In [63]:
# l = []
# for ch in text:
#     if ch == '0':
#         l.append('0000')
#     else:
#         s = '0'*int(4-ceil(log2(int(ch,16))+0.001)) + bin(int(ch,16))[2:]
#         l.append(s)
# h2b('ab5')

**binary-to-hex**

In [64]:
def b2h(binary):
    bits = [binary[i]+binary[i+1]+binary[i+2]+binary[i+3] for i in range(0,len(binary),4)]
    return ''.join([f'{hex(int(num,2))[2:]}' for num in bits])

**calculating Hamming distance to compare between plain text and cipher text**

In [65]:
def hamming_distance(binary_text1,binary_text2):
    return sum([int(a)^int(b) for a,b in zip(binary_text1,binary_text2)])

**indexing the primitive factors for multiplication in Galois field**

In [66]:
def get_indices(hex_text):
    return [7-i for i,bit in enumerate(h2b(hex_text)) if int(bit)]

**convert indices to binary form**

In [67]:
def indices2binary(indices):
    return ''.join(['1' if i in indices else '0' for i in range(7,-1,-1)])

**segment the text into pieces of 128 characters**

In [68]:
def text_segmentation(text):
    corpus = [text[i:i+16] for i in range(0,len(text),16)]
    tail = corpus[-1]
    corpus[-1] += '#'*(16-len(corpus[-1]))

    print(corpus)
    return corpus

**matrix structure**

In [69]:
def text_matrix(text):
    text = [text[i:i+2] for i in range(0,len(text),2)]
    matrix = np.transpose(np.reshape(np.array(text),(4,4)))
    return matrix

**XOR of 2 hex number**

In [70]:
def xor_hex(first,second):
    return b2h(
            ''.join(
                [''.join(
                    [
                        str(int(a)^int(b)) for a,b in zip(h2b(f_values),h2b(s_values))]) 
                         for f_values,s_values in zip(first,second)]
                    )
            )

**add round key step**

In [71]:
def add_round_key(text,key):
    for i in range(4):
        for j in range(4):
            text[i,j] = xor_hex(text[i,j],key[i,j])
    return text

**subbyte transform step**

In [72]:
def sub_byte_transform(text):
    for i in range(4):
        for j in range(4):
            row,col = text[i,j]
            text[i,j] = s_box[int(row,16),int(col,16)]
    return text

**shift rows step**

In [73]:
def shift_rows(text):
    text = np.ndarray.tolist(text)
    for i in range(4):
        for j in range(i):
            text[i].append(text[i].pop(0))
    return np.array(text)

**mixed columns step**

In [75]:
def mixed_columns(text):
    for j in range(4):
        col = text[:,j]
        new_col = []
        for r in range(4):
            row = constant_matrix[r,:]
            products = [product_galois_field(col[i],row[i]) for i in range(4)]
            result = '00'
            for p in products:
                result = xor_hex(result,p)
            new_col.append(result)
        text[:,j] = np.array(new_col)
    return text

**Galois field multiplication for mixed column step**

In [74]:
def product_galois_field(a,b):
    a = get_indices(a)
    b = get_indices(b)
    if len(a) == 0 or len(b) == 0:
        return '00'
    product_indices = []

    for a_index in a:
        for b_index in b:
            if a_index + b_index in product_indices:
                product_indices.remove(a_index + b_index)
            else:
                product_indices.append(a_index + b_index)
    product_indices.sort(reverse=True)
    coefs = [8,4,3,1,0]
    while product_indices[0] >= 8:
        greatest = product_indices[0]
        index = greatest - 8
        for idx in coefs:
            if idx + index in product_indices:
                product_indices.remove(idx + index)
            else:
                product_indices.append(idx + index)
        product_indices.sort(reverse=True)

    return b2h(indices2binary(product_indices))

**Encryption round**

In [76]:
def encrypt_round(text,key,final=False):
    text = sub_byte_transform(text)
    text = shift_rows(text)
    if not final:
        text = mixed_columns(text)
    text = add_round_key(text,key)
    return text

**calculating t_i in key expansion**

In [77]:
def key_changer(col,round):
    col = np.ndarray.tolist(col)
    col.append(col.pop(0))
    for i,item in enumerate(col):
        r,c = item
        col[i] = s_box[int(r,16),int(c,16)]
    const = [Rconstant[round-1][i:i+2] for i in range(0,len(Rconstant[i-1]),2)]
    for i,item in enumerate(col):
        col[i] = xor_hex(item,const[i])
    return np.array(col)

**return the matrix based on the columns to the output.**

In [78]:
def columnwise(matrix):
    matrix = matrix.transpose()
    matrix = np.reshape(matrix,16)
    matrix = np.ndarray.tolist(matrix)
    return ''.join(matrix)

**key expansion**

In [79]:
def key_expansion(key):
    subkey = []
    key = text_matrix(key)
    subkey.append(key.copy())
    
    for round in range(1,11):
        last_col = key[:,3]
        last_col = key_changer(last_col,round)
        for i in range(4):
            key[i,0] = xor_hex(last_col[i],key[i,0])
        for j in range(1,4):
            for i in range(4):
                key[i,j] = xor_hex(key[i,j],key[i,j-1])
        subkey.append(key.copy())

    return subkey

**Encryption**

In [80]:
def encryption(text,key):
    text = text_matrix(t2h(text))
    keys = key_expansion(t2h(key))
    text = add_round_key(text,keys[0])
    for i in range(1,11):
        if i == 10:
            text = encrypt_round(text,keys[i],final=True)
        else:
            text = encrypt_round(text,keys[i])
    return text

# **Define constants**

In [102]:
plain_text = 'cryptography#project#example#one#saaghi'
original_key = 'this#is#the#key#'

In [103]:
s_box = '637c777bf26b6fc53001672bfed7ab76ca82c97dfa5947f0add4a2af9ca472c0b7fd9326363ff7cc34a5e5f171d8311504c723c31896059a071280e2eb27b27509832c1a1b6e5aa0523bd6b329e32f8453d100ed20fcb15b6acbbe394a4c58cfd0efaafb434d338545f9027f503c9fa851a3408f929d38f5bcb6da2110fff3d2cd0c13ec5f974417c4a77e3d645d197360814fdc222a908846eeb814de5e0bdbe0323a0a4906245cc2d3ac629195e479e7c8376d8dd54ea96c56f4ea657aae08ba78252e1ca6b4c6e8dd741f4bbd8b8a703eb5664803f60e613557b986c11d9ee1f8981169d98e949b1e87e9ce5528df8ca1890dbfe6426841992d0fb054bb16'
s_box = np.reshape(np.array([s_box[i:i+2] for i in range(0,len(s_box),2)]),(16,16))

In [104]:
constant_matrix = '02030101010203010101020303010102'
constant_matrix = np.reshape(np.array([constant_matrix[i:i+2] for i in range(0,len(constant_matrix),2)]),(4,4))
print(constant_matrix)

[['02' '03' '01' '01']
 ['01' '02' '03' '01']
 ['01' '01' '02' '03']
 ['03' '01' '01' '02']]


In [105]:
Rconstant = '01000000020000000400000008000000100000002000000040000000800000001b00000036000000'
Rconstant = np.array([Rconstant[i:i+8] for i in range(0,len(Rconstant),8)])
print(Rconstant)

['01000000' '02000000' '04000000' '08000000' '10000000' '20000000'
 '40000000' '80000000' '1b000000' '36000000']


# **Some Example from functions that implemented in previous section**

**Primitive indexing**

In [106]:
pows = get_indices('83')
print(pows)
print(indices2binary(pows))

[7, 1, 0]
10000011


**Hamming distance**

In [107]:
t1 = h2b(t2h('qw'))
t2 = h2b(t2h('mt'))
print(t1)
print(t2)
print(hamming_distance(t1,t2))


0001000000010110
0000110000010011
5


**converting**

In [108]:
hex_text = t2h(plain_text)
binary_text = h2b(hex_text)

In [109]:
print(plain_text)
print(hex_text)
print(binary_text)
print(b2h(binary_text))
print(h2t(b2h(binary_text)))

cryptography#project#example#one#saaghi
0211180f130e0611000f07181a0f110e090402131a0417000c0f0b041a0e0d041a120000060708
000000100001000100011000000011110001001100001110000001100001000100000000000011110000011100011000000110100000111100010001000011100000100100000100000000100001001100011010000001000001011100000000000011000000111100001011000001000001101000001110000011010000010000011010000100100000000000000000000001100000011100001000
0211180f130e0611000f07181a0f110e090402131a0417000c0f0b041a0e0d041a120000060708
cryptography#project#example#one#saaghi


**segmentation**

In [110]:
segments = text_segmentation(plain_text)
for s in segments:
    print(len(h2b(t2h(s))))

['cryptography#pro', 'ject#example#one', '#saaghi#########']
128
128
128


**create main matrix**

In [90]:
text_matrix(t2h(segments[0]))

array([['02', '13', '00', '1a'],
       ['11', '0e', '0f', '0f'],
       ['18', '06', '07', '11'],
       ['0f', '11', '18', '0e']], dtype='<U2')

**Galois field multiplication**

In [112]:
print(product_galois_field('0e','04'))
print(product_galois_field('55','aa'))
print(product_galois_field('af','08'))
print(product_galois_field('02','d4'))

38
59
0f
b3


**XOR**

In [92]:
text = text_matrix(t2h(segments[0]))
print(f'text\n{text}')
print('-'*25)
key = text_matrix(t2h(segments[1]))
print(f'key\n{key}')
print('-'*25)
text = add_round_key(text,key)
print(f'XOR text and key\n{text}')

text
[['02' '13' '00' '1a']
 ['11' '0e' '0f' '0f']
 ['18' '06' '07' '11']
 ['0f' '11' '18' '0e']]
-------------------------
key
[['09' '1a' '0c' '1a']
 ['04' '04' '0f' '0e']
 ['02' '17' '0b' '0d']
 ['13' '00' '04' '04']]
-------------------------
XOR text and key
[['0b' '09' '0c' '00']
 ['15' '0a' '00' '01']
 ['1a' '11' '0c' '1c']
 ['1c' '11' '1c' '0a']]


**subbyte transform**

In [93]:
print(text)
print('-'*25)
text = sub_byte_transform(text)
print(text)

[['0b' '09' '0c' '00']
 ['15' '0a' '00' '01']
 ['1a' '11' '0c' '1c']
 ['1c' '11' '1c' '0a']]
-------------------------
[['2b' '01' 'fe' '63']
 ['59' '67' '63' '7c']
 ['a2' '82' 'fe' '9c']
 ['9c' '82' '9c' '67']]


**shift rows**

In [94]:
print(text)
print('-'*25)
text = shift_rows(text)
print(text)

[['2b' '01' 'fe' '63']
 ['59' '67' '63' '7c']
 ['a2' '82' 'fe' '9c']
 ['9c' '82' '9c' '67']]
-------------------------
[['2b' '01' 'fe' '63']
 ['67' '63' '7c' '59']
 ['fe' '9c' 'a2' '82']
 ['67' '9c' '82' '9c']]


**mix columns**

In [113]:
sample = 'd4e0b81ebfb441275d52119830aef1e5'
sample = np.reshape(np.array([sample[i:i+2] for i in range(0,len(sample),2)]),(4,4))
print(sample)
print('-'*25)
sample = mixed_columns(sample)
print(sample)

[['d4' 'e0' 'b8' '1e']
 ['bf' 'b4' '41' '27']
 ['5d' '52' '11' '98']
 ['30' 'ae' 'f1' 'e5']]
-------------------------
[['04' 'e0' '48' '28']
 ['66' 'cb' 'f8' '06']
 ['81' '19' 'd3' '26']
 ['e5' '9a' '7a' '4c']]


**key expansion**

In [96]:
ori_key = '2b7e151628aed2a6abf7158809cf4f3c'
keys = key_expansion(ori_key)
# print(*keys)
for i,k in enumerate(keys):
    print(f'key({i}) = {columnwise(k)}')

key(0) = 2b7e151628aed2a6abf7158809cf4f3c
key(1) = a0fafe1788542cb123a339392a6c7605
key(2) = f2c295f27a96b9435935807a7359f67f
key(3) = 3d80477d4716fe3e1e237e446d7a883b
key(4) = ef44a541a8525b7fb671253bdb0bad00
key(5) = d4d1c6f87c839d87caf2b8bc11f915bc
key(6) = 6d88a37a110b3efddbf98641ca0093fd
key(7) = 4e54f70e5f5fc9f384a64fb24ea6dc4f
key(8) = ead27321b58dbad2312bf5607f8d292f
key(9) = ac7766f319fadc2128d12941575c006e
key(10) = d014f9a8c9ee2589e13f0cc8b6630ca6


# **RUN AES**

**encryption**

In [114]:
print(segments[0])
cipher_text = encryption(segments[0],original_key)
cipher_text = columnwise(cipher_text)
# print(cipher_text)
print(f'cifer text = {h2t(cipher_text)}')
print(f'cifer text in hex = {cipher_text}')

cryptography#pro
cifer text = ĳÃ²çfÌó}ĆĉŁ´ăŃŎ
cifer text in hex = d2625186056b921ca5a8e053a2e22ced


**calculate hamming distance between the plain text and cipher text to understand how many bits were changed!**

In [98]:
print(h2b(t2h(segments[0])))
print(h2b(cipher_text))
print(hamming_distance(h2b(t2h(segments[0])),h2b(cipher_text)))

00000010000100010001100000001111000100110000111000000110000100010000000000001111000001110001100000011010000011110001000100001110
11010010011000100101000110000110000001010110101110010010000111001010010110101000111000000101001110100010111000100010110011101101
66
